In [8]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd

#Move indicator into its own Class so that will show up in output file. 
#Reference: https://www.backtrader.com/docu/inddev.html


class Bollinger_Breakout(bt.Indicator):
    lines = ('breakout',)
    
    def __init__(self):
        self.lines.breakout = self.data > bt.indicators.BBands(self.data, period=20).lines.top
        
class Bollinger_Breakdown(bt.Indicator):
    lines = ('breakdown',)
    
    def __init__(self):
        self.lines.breakdown = self.data < bt.indicators.BBands(self.data, period=20).lines.bot


class RSI_Overbought(bt.Indicator):
    lines = ('overbought',)
    
    params = (
        ('level', 70),    
    )

    def __init__(self):
        self.lines.overbought = bt.indicators.RSI(self.data) > self.p.level
        
class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level
        print(bt.indicators.RSI(self.data))
        

# Create a Stratey to trade hv
class HVStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7),
        ("y", None)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
      
        # Keep a reference to the close price, historical vol as well as returns
        self.stock_px = self.datas[symbols.index("stockpx")].close
        self.vol = self.datas[symbols.index(y)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
        self.bband = bt.indicators.BBands(self.stock_px, period=20)
        self.sma_short = bt.indicators.SimpleMovingAverage(self.stock_px, period=50)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.stock_px, period=200)
        self.dsma = self.sma_short - self.sma_long
        self.mom = bt.Cmp(self.sma_short, self.sma_long)
        #self.rsi_overbought = bt.indicators.RSI(self.stock_px) > 70
        #self.rsi_oversold = bt.indicators.RSI(self.stock_px) <30
        #self.bollinger_breakout = self.stock_px > self.bband.lines.top
        #self.bollinger_breakdown = self.stock_px < self.bband.lines.bot

        
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        self.rsi_overbought = RSI_Overbought(self.stock_px)
        self.rsi_oversold = RSI_Oversold(self.stock_px)

        #self.rsi_oversold = bt.indicators.RSI(self.stock_px) <30
        
        self.bollinger_breakout = Bollinger_Breakout(self.stock_px)
        self.bollinger_breakdown = Bollinger_Breakdown(self.stock_px)


        self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
        self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)

        self.in_trade = False
        self.days_in_trade = 0
        
        
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f; Vol, %.2f' % (self.stock_px[0], self.vol[0]))
        
        size = self.position.size
        
        # Check if we are currently in a trade
        if(not size):
            #If not has our indicator triggered?
            if self.bollinger_breakout[0] > 0 and self.rsi_overbought[0] > 0:
                print("Breakout & Overbought indicators triggered...entering position")
                self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
                
            elif self.bollinger_breakdown[0] > 0 and self.rsi_oversold[0] > 0:
                print("Breakdown & Oversold indicators triggered...entering position")
                self.buy(exectype=bt.Order.Close)
                self.days_in_trade = 1
            
        else:
            self.days_in_trade += 1
            print("In position %d days" % self.days_in_trade)
            if self.days_in_trade == self.p.max_days_in_trade:
                print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
        
        '''
        if(not size):
            #If not has our indicator triggered?
            if self.rsi_oversold[0] > 0:
                print("Oversold indicator triggered...entering position")
                self.buy(exectype=bt.Order.Close)
                self.days_in_trade = 1
        else:
            self.days_in_trade += 1
            print("In position %d days" % self.days_in_trade)
            if self.days_in_trade == self.p.max_days_in_trade:
                print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
                
        if(not size):
            #If not has our indicator triggered?
            if self.rsi_overbought[0] > 0:
                print("Overbought indicator triggered...entering position")
                self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
        else:
            self.days_in_trade += 1
            print("In position %d days" % self.days_in_trade)
            if self.days_in_trade == self.p.max_days_in_trade:
                print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
        '''
                

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    # Add data
    datapath = os.path.join('../../../datas/YHOO.csv')
    
    datas = pd.read_csv(datapath, parse_dates=True,index_col=0)
    
    y = '10dORHV' #iv30
    symbol_cols = [y,'stockpx'] #symbol we're trying to forecast/trade first
    #symbol_cols = list(datas) #Load all symbols
    
    
    # Add the Data Feeds to Cerebro
    for i, symbol in enumerate(symbol_cols):
        data = datas[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add a strategy
    cerebro.addstrategy(HVStrategy, symbols=symbol_cols, y=y)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='hv-strategy_RSI&BBands.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2010-01-04, Close, 17.10; Vol, 24.69
2010-01-05, Close, 17.22; Vol, 23.59
2010-01-06, Close, 17.16; Vol, 22.59
2010-01-07, Close, 16.70; Vol, 24.71
2010-01-08, Close, 16.70; Vol, 20.74
2010-01-11, Close, 16.80; Vol, 17.82
2010-01-12, Close, 16.69; Vol, 17.62
2010-01-13, Close, 16.88; Vol, 17.68
2010-01-14, Close, 17.17; Vol, 19.14
2010-01-15, Close, 16.85; Vol, 20.49
2010-01-19, Close, 16.74; Vol, 18.94
2010-01-20, Close, 16.40; Vol, 19.92
2010-01-21, Close, 16.22; Vol, 20.31
2010-01-22, Close, 15.86; Vol, 19.38
2010-01-25, Close, 15.88; Vol, 20.83
2010-01-26, Close, 15.98; Vol, 22.32
2010-01-27, Close, 15.96; Vol, 25.89
2010-01-28, Close, 15.48; Vol, 28.39
2010-01-29, Close, 15.08; Vol, 29.18
2010-02-01, Close, 15.05; Vol, 28.82
Breakdown & Oversold indicators triggered...entering position
2010-02-02, Close, 15.16; Vol, 29.59
In position 2 days
2010-02-03, Close, 15.46; Vol, 28.74
In position 3 days
2010-02-04, Close, 15.04; Vol, 31.07
In position 4

2011-08-09, Close, 12.00; Vol, 57.62
In position 4 days
2011-08-10, Close, 11.91; Vol, 58.52
In position 5 days
2011-08-11, Close, 12.90; Vol, 67.42
In position 6 days
2011-08-12, Close, 13.38; Vol, 66.76
In position 7 days
Max days to hold trade reached...closing position
2011-08-15, Close, 13.47; Vol, 66.36
2011-08-16, Close, 13.51; Vol, 65.93
2011-08-17, Close, 13.46; Vol, 65.10
2011-08-18, Close, 12.98; Vol, 60.60
2011-08-19, Close, 12.98; Vol, 61.63
2011-08-22, Close, 12.82; Vol, 61.41
2011-08-23, Close, 13.29; Vol, 59.41
2011-08-24, Close, 13.11; Vol, 57.90
2011-08-25, Close, 12.91; Vol, 48.25
2011-08-26, Close, 12.75; Vol, 37.42
2011-08-29, Close, 13.65; Vol, 48.50
2011-08-30, Close, 13.90; Vol, 57.07
2011-08-31, Close, 13.71; Vol, 57.55
2011-09-01, Close, 13.36; Vol, 55.67
2011-09-02, Close, 12.92; Vol, 59.21
2011-09-06, Close, 12.93; Vol, 58.67
2011-09-07, Close, 13.61; Vol, 64.95
2011-09-08, Close, 14.35; Vol, 70.12
2011-09-09, Close, 14.51; Vol, 69.37
2011-09-12, Close, 14.2

2013-04-11, Close, 24.45; Vol, 17.57
2013-04-12, Close, 24.76; Vol, 17.65
2013-04-15, Close, 24.06; Vol, 24.38
2013-04-16, Close, 23.94; Vol, 27.10
2013-04-17, Close, 23.75; Vol, 26.33
2013-04-18, Close, 23.26; Vol, 30.14
2013-04-19, Close, 23.60; Vol, 30.33
2013-04-22, Close, 23.88; Vol, 31.13
2013-04-23, Close, 24.37; Vol, 31.71
2013-04-24, Close, 24.76; Vol, 31.62
2013-04-25, Close, 25.13; Vol, 32.01
2013-04-26, Close, 24.76; Vol, 32.23
2013-04-29, Close, 24.45; Vol, 30.27
2013-04-30, Close, 24.69; Vol, 29.78
2013-05-01, Close, 24.29; Vol, 28.84
2013-05-02, Close, 24.96; Vol, 26.22
2013-05-03, Close, 25.09; Vol, 26.33
2013-05-06, Close, 25.15; Vol, 25.19
2013-05-07, Close, 26.02; Vol, 43.19
2013-05-08, Close, 26.47; Vol, 43.39
Breakout & Overbought indicators triggered...entering position
2013-05-09, Close, 26.20; Vol, 43.20
In position 2 days
2013-05-10, Close, 26.76; Vol, 42.70
In position 3 days
2013-05-13, Close, 26.42; Vol, 42.41
In position 4 days
2013-05-14, Close, 26.72; Vol

2014-11-05, Close, 47.50; Vol, 29.45
In position 7 days
Max days to hold trade reached...closing position
2014-11-06, Close, 47.81; Vol, 30.17
2014-11-07, Close, 48.59; Vol, 29.16
2014-11-10, Close, 49.42; Vol, 26.26
2014-11-12, Close, 50.59; Vol, 27.61
2014-11-13, Close, 50.52; Vol, 29.02
2014-11-14, Close, 51.66; Vol, 28.38
2014-11-17, Close, 52.03; Vol, 28.88
2014-11-18, Close, 51.79; Vol, 26.03
2014-11-19, Close, 50.66; Vol, 26.88
2014-11-20, Close, 51.24; Vol, 27.85
2014-11-21, Close, 51.05; Vol, 30.37
2014-11-24, Close, 51.62; Vol, 30.48
2014-11-25, Close, 51.76; Vol, 29.54
2014-11-26, Close, 51.97; Vol, 27.74
2014-11-28, Close, 51.74; Vol, 26.22
2014-12-01, Close, 50.26; Vol, 26.30
2014-12-02, Close, 50.72; Vol, 26.22
2014-12-03, Close, 50.55; Vol, 25.75
2014-12-04, Close, 50.46; Vol, 23.57
2014-12-05, Close, 51.00; Vol, 22.69
2014-12-08, Close, 49.54; Vol, 20.08
2014-12-09, Close, 50.44; Vol, 26.66
2014-12-10, Close, 49.35; Vol, 28.85
2014-12-11, Close, 49.89; Vol, 29.22
2014-1

2016-06-27, Close, 35.12; Vol, 25.48
2016-06-28, Close, 36.06; Vol, 25.03
2016-06-29, Close, 36.99; Vol, 26.10
2016-06-30, Close, 37.44; Vol, 26.02
2016-07-01, Close, 37.98; Vol, 26.15
2016-07-05, Close, 37.36; Vol, 25.57
2016-07-06, Close, 37.44; Vol, 25.28
2016-07-07, Close, 37.50; Vol, 25.44
2016-07-08, Close, 37.50; Vol, 27.12
2016-07-11, Close, 38.10; Vol, 22.10
2016-07-12, Close, 37.94; Vol, 20.07
2016-07-13, Close, 37.63; Vol, 18.30
2016-07-14, Close, 38.06; Vol, 17.03
2016-07-15, Close, 37.81; Vol, 16.59
2016-07-18, Close, 37.91; Vol, 15.45
2016-07-19, Close, 38.17; Vol, 16.16
2016-07-20, Close, 38.61; Vol, 17.28
2016-07-21, Close, 38.82; Vol, 17.47
2016-07-22, Close, 39.35; Vol, 15.85
2016-07-25, Close, 38.40; Vol, 16.05
2016-07-26, Close, 38.83; Vol, 17.69
2016-07-27, Close, 38.70; Vol, 17.62
2016-07-28, Close, 38.45; Vol, 17.34
2016-07-29, Close, 38.16; Vol, 16.96
2016-08-01, Close, 38.80; Vol, 18.58
2016-08-02, Close, 38.59; Vol, 17.58
2016-08-03, Close, 38.36; Vol, 16.63
2